In [14]:
class Classifer:
    def __init__(self,filename,dataFormat):
        '''
        filename  文件名
        dataFormat 数据文件格式列表
        '''
        total=0
        classes={}  #存类别出现的次数  为先验概率做准备
        counts={}   #每个类别对应的属性值出现的次数  为条件概率做准备
        self.format=dataFormat.strip().split('\t')  #表示格式  数组
        self.prior={}  #先验概率
        self.condition={}  #条件概率
        
        #构建文件名
        f=open(filename,'r')
        lines=f.readlines()
        f.close()
        
        colValueCount={}    #{1:2,2:2,3:2,....}
        colValues={}
        
        for line in lines:
            total=total+1  #总记录数
            fields=line.strip().split('\t')   #切分每一条数据
            
            vector=[]   #存原始数据的位置
            for i in range( len(fields) ):
                #判断这一列是什么类型的值
                if self.format[i]=='attr':
                    vector.append(fields[i])
                elif self.format[i]=='class':
                    category=fields[i]
                    
            #记录每个类别出现的次数
            classes.setdefault(category,0)
            counts.setdefault( category,{})
            #类别数加1
            classes[category]+=1  #{'i100':1,'i500':1}
            
            #处理每条记录出现的属性值
            #循环vector 取出每列值
            col=0
            for columnValue in vector:
                col+=1
                counts[category].setdefault(col,{} )
                counts[category][col].setdefault(columnValue,0 )  #{'i100':{1:{'both':1,'health':1,'apreace':1},2:{'sedentary':1}  }  }
                counts[category][col][columnValue]+=1
                colValues.setdefault(col,set() )
                colValues[col].add(columnValue)
                
        for (col,columnValues)in colValues.items():
            colValueCount.setdefault(col,len(columnValues ) )
            print('第',col,'列有',len(columnValues),'个取值')
                
        #对应的计算成概率        
        #1.先验概率
        for (category,count) in classes.items():
            self.prior[category]=count/total
            
        #2.条件概率   注意 colums是一个字典
        for (category,columns) in counts.items():
            self.condition.setdefault(category,{})
            for (col,valueCount) in columns.items():
                self.condition[category].setdefault(col,{})
                for (attrValue,count) in valueCount.items():
                    #self.condition[category][col][attrValue] = count/classes[category]
                    #m=len(counts[category][col].items())
                    m=colValueCount[col]
                    print('分类',category,'第',col,'列的取值个数有:',m)
                    nc=count
                    n=classes[category]
                    temp=(1+nc)/(m+n)
                    self.condition[category][col][attrValue] = temp                    
        
        #最后将  分类数和条件数值数存到self中
        self.classCounts=classes  #分类计数
        self.conditionCount=counts  #条件计数
        self.colValueCount=colValueCount
        
    def classify(self,itemVector):
        '''
        朴素贝叶斯：求出所有可能性的概率值，取大值
        '''
        self.result=[]  #[ (后验概率，类别1),(后验概率，类别2),.....]
        for (category,priors) in self.prior.items():
            prob=priors  #这个category类别的先验概率
            col=1
            for colValue in itemVector:
                if not colValue in self.condition[category][col]:
                    #print('分类为:',category,'第',col,'列',colValue,'出现了空值')
                    #m=len(self.condition[category][priors].items())
                    m=self.colValueCount[col]
                    temp=1/(m+self.classCounts[category])
                    prob=prob*temp
                else:
                    prob=prob * self.condition[category][col][colValue]
                col+=1
            self.result.append( (prob,category) )  # [(0.3,'i100'), ]
        totalProb=0.0
        print(self.result)
        for (p,cat) in self.result:
            totalProb+=p
            
        self.belief=max(self.result)[0]/totalProb
        return max(self.result)[1]

In [15]:
c=Classifer(  'hv-09', 'class\tattr\tattr\tattr\tattr\tattr\tattr\tattr\tattr\tattr\tattr\tattr\tattr\tattr\tattr\tattr\tattr'  )
print(  c.classCounts )   #各分类出现的次数, 表示在当前数据集中每个分类出现的次数
print( c.conditionCount ) # 各条件出现的次数
# 模型
print(  c.prior )   #先验概率
print(  c.condition ) #条件概率

第 1 列有 2 个取值
第 2 列有 2 个取值
第 3 列有 2 个取值
第 4 列有 2 个取值
第 5 列有 2 个取值
第 6 列有 2 个取值
第 7 列有 2 个取值
第 8 列有 2 个取值
第 9 列有 2 个取值
第 10 列有 2 个取值
第 11 列有 2 个取值
第 12 列有 2 个取值
第 13 列有 2 个取值
第 14 列有 2 个取值
第 15 列有 2 个取值
第 16 列有 2 个取值
分类 democrat 第 1 列的取值个数有: 2
分类 democrat 第 1 列的取值个数有: 2
分类 democrat 第 2 列的取值个数有: 2
分类 democrat 第 2 列的取值个数有: 2
分类 democrat 第 3 列的取值个数有: 2
分类 democrat 第 4 列的取值个数有: 2
分类 democrat 第 5 列的取值个数有: 2
分类 democrat 第 6 列的取值个数有: 2
分类 democrat 第 6 列的取值个数有: 2
分类 democrat 第 7 列的取值个数有: 2
分类 democrat 第 8 列的取值个数有: 2
分类 democrat 第 9 列的取值个数有: 2
分类 democrat 第 9 列的取值个数有: 2
分类 democrat 第 10 列的取值个数有: 2
分类 democrat 第 10 列的取值个数有: 2
分类 democrat 第 11 列的取值个数有: 2
分类 democrat 第 11 列的取值个数有: 2
分类 democrat 第 12 列的取值个数有: 2
分类 democrat 第 13 列的取值个数有: 2
分类 democrat 第 13 列的取值个数有: 2
分类 democrat 第 14 列的取值个数有: 2
分类 democrat 第 14 列的取值个数有: 2
分类 democrat 第 15 列的取值个数有: 2
分类 democrat 第 15 列的取值个数有: 2
分类 democrat 第 16 列的取值个数有: 2
分类 republican 第 1 列的取值个数有: 2
分类 republican 第 1 列的取值个数有: 2
分类 republican 第 2 列的取值个数有: 2
分类 republic

In [16]:
c.classify(['y','n','y','n','y','n','y','n','y','n','y','n','y','n','y','n'])
print(c.result)
print('以上结果的置信度:',c.belief)

[(3.2631051495108007e-07, 'democrat'), (8.17808941137818e-11, 'republican')]
[(3.2631051495108007e-07, 'democrat'), (8.17808941137818e-11, 'republican')]
以上结果的置信度: 0.9997494398760179
